# Graph measures

- Closeness
- Graph diameter
- Average path length
- In-out degree

Graph measures were added in MADlib 1.12.  Some graph measures require a valid output from a prior APSP run - both the APSP table and the associated output summary table must be present.

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum 4.3.10.0
%sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum 4.2.3.0
#%sql postgresql://gpdbchina@10.194.10.68:55000/madlib

u'Connected: gpdbchina@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.12-dev, git revision: rel/v1.11-51-g69f7886, cmake configuration time: Fri Aug 18 16:30:51 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# Closeness

# 1.  Create vertex and edge tables

In [18]:
%%sql 
DROP TABLE IF EXISTS vertex, edge;

CREATE TABLE vertex(
        id INTEGER,
        name TEXT
        );

CREATE TABLE edge(
        src_id INTEGER,
        dest_id INTEGER,
        edge_weight FLOAT8
        );

INSERT INTO vertex VALUES
(0, 'A'),
(1, 'B'),
(2, 'C'),
(3, 'D'),
(4, 'E'),
(5, 'F'),
(6, 'G'),
(7, 'H');

INSERT INTO edge VALUES
(0, 1, 1.0),
(0, 2, 1.0),
(0, 4, 10.0),
(1, 2, 2.0),
(1, 3, 10.0),
(2, 3, 1.0),
(2, 5, 1.0),
(2, 6, 3.0),
(3, 0, 1.0),
(4, 0, -2.0),
(5, 6, 1.0),
(6, 7, 1.0);

SELECT * FROM edge ORDER BY src_id, dest_id;

Done.
Done.
Done.
8 rows affected.
12 rows affected.
12 rows affected.


src_id,dest_id,edge_weight
0,1,1.0
0,2,1.0
0,4,10.0
1,2,2.0
1,3,10.0
2,3,1.0
2,5,1.0
2,6,3.0
3,0,1.0
4,0,-2.0


# 2.  Calculate APSP

In [29]:
%%sql
DROP TABLE IF EXISTS out_apsp, out_apsp_summary;

SELECT madlib.graph_apsp('vertex',      -- Vertex table
                         'id',          -- Vertix id column (NULL means use default naming)
                         'edge',        -- Edge table
                         'src=src_id, dest=dest_id, weight=edge_weight',
                                        -- Edge arguments (NULL means use default naming)
                         'out_apsp');   -- Output table of shortest paths

SELECT * FROM out_apsp ORDER BY src_id, dest_id;

Done.
1 rows affected.
64 rows affected.


src_id,dest_id,edge_weight,parent
0,0,0.0,0
0,1,1.0,1
0,2,1.0,2
0,3,2.0,2
0,4,10.0,4
0,5,2.0,2
0,6,3.0,5
0,7,4.0,6
1,0,4.0,3
1,1,0.0,1


# 3.  Compute the closeness measure for all nodes

In [20]:
%%sql
DROP TABLE IF EXISTS out_closeness;
SELECT madlib.graph_closeness('out_apsp', 'out_closeness');
SELECT * FROM out_closeness ORDER BY src_id;

Done.
1 rows affected.
8 rows affected.


src_id,inverse_sum_dist,inverse_avg_dist,sum_inverse_dist,k_degree
0,0.0434782608696,0.304347826087,3.68333333333,7
1,0.0285714285714,0.2,1.9380952381,7
2,0.0416666666667,0.291666666667,3.75,7
3,0.0357142857143,0.25,2.87424242424,7
4,-1.0,-7.0,-1.0,7
5,0.333333333333,0.666666666667,1.5,2
6,1.0,1.0,1.0,1
7,None,None,0.0,0


# 4.  Create a graph with 2 groups

In [31]:
%%sql 
DROP TABLE IF EXISTS edge_gr;

CREATE TABLE edge_gr AS
(
  SELECT *, 0 AS grp FROM edge
  UNION
  SELECT *, 1 AS grp FROM edge WHERE src_id < 6 AND dest_id < 6
);

INSERT INTO edge_gr VALUES
(4,5,-20,1);

SELECT * FROM edge_gr ORDER BY grp, src_id, dest_id;

Done.
21 rows affected.
1 rows affected.
22 rows affected.


src_id,dest_id,edge_weight,grp
0,1,1.0,0
0,2,1.0,0
0,4,10.0,0
1,2,2.0,0
1,3,10.0,0
2,3,1.0,0
2,5,1.0,0
2,6,3.0,0
3,0,1.0,0
4,0,-2.0,0


# 5.  Find APSP for all groups

In [32]:
%%sql
DROP TABLE IF EXISTS out_gr, out_gr_summary;

SELECT madlib.graph_apsp(
                         'vertex',      -- Vertex table
                         NULL,          -- Vertex id column (NULL means use default naming)
                         'edge_gr',     -- Edge table
                         'src=src_id, dest=dest_id, weight=edge_weight',
                         'out_gr',      -- Output table of shortest paths
                         'grp'          -- Grouping columns
);

SELECT * FROM out_gr ORDER BY grp, src_id, dest_id;

Done.
1 rows affected.
100 rows affected.


grp,src_id,dest_id,edge_weight,parent
0,0,0,0.0,0
0,0,1,1.0,1
0,0,2,1.0,2
0,0,3,2.0,2
0,0,4,10.0,4
0,0,5,2.0,2
0,0,6,3.0,5
0,0,7,4.0,6
0,1,0,4.0,3
0,1,1,0.0,1


# 6. Compute closeness measure for vertex 0 to vertex 5 in every group

In [24]:
%%sql
DROP TABLE IF EXISTS out_gr_closeness;

SELECT madlib.graph_closeness(
    'out_gr', 
    'out_gr_closeness', 
    'src_id >= 0 and src_id <=5');

SELECT * FROM out_gr_closeness ORDER BY grp, src_id;

Done.
1 rows affected.
12 rows affected.


grp,src_id,inverse_sum_dist,inverse_avg_dist,sum_inverse_dist,k_degree
0,0,0.0434782608696,0.304347826087,3.68333333333,7
0,1,0.0285714285714,0.2,1.9380952381,7
0,2,0.0416666666667,0.291666666667,3.75,7
0,3,0.0357142857143,0.25,2.87424242424,7
0,4,-1.0,-7.0,-1.0,7
0,5,0.333333333333,0.666666666667,1.5,2
1,0,0.25,1.25,2.5,5
1,1,0.0588235294118,0.294117647059,0.988095238095,5
1,2,0.1,0.5,1.79166666667,5
1,3,0.142857142857,0.714285714286,1.9797979798,5


#  Graph Diameter
Use the same graph and APSP output from above.

In [26]:
%%sql
DROP TABLE IF EXISTS out_diameter;
SELECT madlib.graph_diameter('out_apsp', 'out_diameter');
SELECT * FROM out_diameter;

Done.
1 rows affected.
1 rows affected.


diameter,diameter_end_vertices
14.0,"[[1, 4]]"


For grouping, use the same graph and APSP output with grouping from above.

In [28]:
%%sql
DROP TABLE IF EXISTS out_gr_path;
SELECT madlib.graph_diameter('out_gr', 'out_gr_diameter');
SELECT * FROM out_gr_diameter ORDER BY grp;

Done.
1 rows affected.
2 rows affected.


grp,diameter,diameter_end_vertices
0,14.0,"[[1, 4]]"
1,14.0,"[[1, 4]]"


#  Average Path Length
Use the same graph and APSP output from above.

In [35]:
%%sql
DROP TABLE IF EXISTS out_avg_path_length;
SELECT madlib.graph_avg_path_length('out_apsp', 'out_avg_path_length');
SELECT * FROM out_avg_path_length;

Done.
1 rows affected.
1 rows affected.


avg_path_length
2.01785714286


For grouping, use the same graph and APSP output with grouping from above.

In [37]:
%%sql
DROP TABLE IF EXISTS out_gr_path;
SELECT madlib.graph_avg_path_length('out_gr', 'out_gr_path');
SELECT * FROM out_gr_path ORDER BY grp;

Done.
1 rows affected.
2 rows affected.


grp,avg_path_length
0,2.01785714286
1,0.466666666667


#  In-Out Degree
Use the same graph and APSP output from above.

In [39]:
%%sql
DROP TABLE IF EXISTS degrees;

SELECT madlib.graph_vertex_degrees(
    'vertex',      -- Vertex table
    'id',          -- Vertix id column (NULL means use default naming)
    'edge',        -- Edge table
    'src=src_id, dest=dest_id, weight=edge_weight',
    'degrees');    -- Output table of shortest paths

SELECT * FROM degrees ORDER BY id;

Done.
1 rows affected.
7 rows affected.


id,indegree,outdegree
0,2,3
1,1,2
2,2,3
3,2,1
4,1,1
5,1,1
6,2,1


For grouping, use the same graph with grouping from above.

In [42]:
%%sql 
DROP TABLE IF EXISTS out_gr;

SELECT madlib.graph_vertex_degrees(
    'vertex',      -- Vertex table
    NULL,          -- Vertex id column (NULL means use default naming)
    'edge_gr',     -- Edge table
    'src=src_id, dest=dest_id, weight=edge_weight',
    'out_gr',      -- Output table of shortest paths
    'grp'          -- Grouping columns
);

SELECT * FROM out_gr WHERE id < 2 ORDER BY grp, id;

Done.
1 rows affected.
4 rows affected.


grp,id,indegree,outdegree
0,0,2,3
0,1,1,2
1,0,2,3
1,1,1,2
